In [1]:
from local.node import Node
from local.constnode import ConstNode
from local.varnode import VarNode
from local.funcnode import FuncNode
from local.func import *
from local.treeoptimizer import TreeOptimizer
from local.crossover import *
from local.Optimizer import *

import os
import json
import glob
import copy
import random
import pickle
import graphviz
import datetime
from PIL import Image
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, losses, metrics, optimizers

In [2]:
def create_folder():
    date = datetime.datetime.today()
    folder_path = str(date.year)
    if not os.path.exists(folder_path):
        os.mkdir(folder_path)

    folder_path += "/"+str(date.month) +"_"+str(date.day)
    if not os.path.exists(folder_path):
        os.mkdir(folder_path)

    folder_path += "/"+str(date.hour)+"_"+str(date.minute)
    if not os.path.exists(folder_path):
        os.mkdir(folder_path)
        
    return folder_path


def Tounament_select(score_list, generation, Selection_size):
    candidate = random.sample(range(len(score_list)), Selection_size)
    candidate_score = [score_list[i] for i in candidate]
    index = candidate_score.index(max(candidate_score))
    return copy.deepcopy(generation[candidate[index]])

In [3]:
Population = 50
Max_generation = 500

Mutation_rate = 0.2
Crossover_rate = 0.8

Selection_size = 2

BATCH_SIZE = 512
EPOCHS = 10

loss_obj = metrics.Mean()
accuracy_obj = metrics.SparseCategoricalAccuracy()
val_loss_obj = metrics.SparseCategoricalCrossentropy()
val_accuracy_obj = metrics.SparseCategoricalAccuracy()

generation = []
generation += [Momentum(learning_rate=0.01) for _ in range(Population // 2)]
generation += [RMSProp() for _ in range(Population // 2)]

best_optim = SGD()
best_score = 0.0
best_score_list = []
past_progress = 0
folder_path = create_folder()
if not os.path.exists(folder_path+"/best_optim/"):
    os.mkdir(folder_path+"/best_optim/")
score_progress = []

In [4]:
def load_data():
    (x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()
    x_train, x_test = x_train[..., tf.newaxis] / 255.0, x_test[..., tf.newaxis] / 255.0

    train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(BATCH_SIZE)
    test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

    return train_dataset, test_dataset

def build_model():
    input = layers.Input(shape = (32, 32, 3))
    x = layers.Conv2D(32, 3, 2)(input)
    x = layers.LeakyReLU()(x)
    x = layers.Flatten()(x)
    x = layers.Dense(units = 128, activation = "relu")(x)
    output = layers.Dense(units = 10, activation = "softmax")(x)

    return models.Model(input, output)

train_dataset, test_dataset = load_data()

model = build_model()
model.compile()
if not os.path.exists("models/"):
    os.mkdir("models/")
if not os.path.exists("models/model_cifar.keras"):
    model.save_weights("models/model_cifar.keras")

In [5]:
# best_score = 0.6003
# folder_path = '2024/1_2/10_4'
# read_folder_path = folder_path +f"/generation{past_progress}"
# pickle_list = glob.glob(read_folder_path +"/*.pickle")
# generation = []
# for path in pickle_list:
#     pickle_file = open(path, "rb")
#     genetic = pickle.load(pickle_file)
#     generation.append(genetic)

In [6]:
for progress in range(past_progress, Max_generation):
    fitness_list = []
    save_path = folder_path + f"/generation{progress}/"
    if not os.path.exists(save_path):
        os.mkdir(save_path)

    for index in tqdm(range(len(generation))):

        acc_list = []
        optimizer = generation[index]
        optimizer.model_params = list()
        optimizer.grads_params = list()
        model.load_weights("models/model_cifar.keras")
        penalty = 1e-5 * len(optimizer.make_struct_dict().keys())

        @tf.function
        def train_step(X, Y):
            with tf.GradientTape() as tape:
                pred = model(X)
                loss = losses.SparseCategoricalCrossentropy()(Y, pred)
            gradients = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))
            
            loss_obj(loss)
            accuracy_obj(Y, pred)
        
            return gradients
        
        @tf.function
        def test_step(X, Y):
            pred = model(X)
            
            val_loss_obj(Y, pred)
            val_accuracy_obj(Y, pred)

        for i in range(EPOCHS):
            
            loss_obj.reset_state()
            accuracy_obj.reset_state()
            val_loss_obj.reset_state()
            val_accuracy_obj.reset_state()
            gradients = 0
            for X, Y in train_dataset:
                gradients = train_step(X, Y)
            
            optimizer.model_params.append(copy.deepcopy(model.trainable_variables))
            optimizer.grads_params.append(copy.deepcopy(gradients))
            
            for X, Y in test_dataset:
                test_step(X, Y)
        
            acc_list.append(val_accuracy_obj.result().numpy())
        fitness_list.append(max(acc_list))
        # 画像保存に時間がかかったのでコメントアウトしてます
        # optimizer.plot_struct(f"{save_path}/optimizer_{index}_{max(acc_list)}")
        pickle_file = open(f"{save_path}/optimizer_{index}_{max(acc_list)}.pickle", mode= "wb")
        pickle.dump(optimizer, pickle_file)
        pickle_file.close()


    for index in range(len(fitness_list)):
        if fitness_list[index] > best_score:
            best_score = fitness_list[index]
            best_optim = copy.deepcopy(generation[index])

            best_optim.plot_struct(f"{folder_path}/best_optim/generation_{progress}_{index}_{best_score}")
            pickle_file = open(f"{folder_path}/best_optim/generation_{progress}_{index}_{best_score}.pickle", mode= "wb")
            pickle.dump(best_optim, pickle_file)
            pickle_file.close()
    
    best_score_list.append(best_score)
    print(best_score)
    json_file = open(f"{folder_path}/best_optim/score{progress}.json", "w")
    json.dump(str(best_score), json_file)
    json_file.close()

    next_generation = []

    # while len(next_generation) < Population:
    #     parent1 = Tounament_select(fitness_list, generation, Selection_size)
    #     parent2 = Tounament_select(fitness_list, generation, Selection_size)

    #     if random.random() < Crossover_rate:
    #         parent1, parent2 = semantic_crossover(parent1, parent2, progress, Max_generation)
        
    #     next_generation.append(parent1)
    #     next_generation.append(parent2)
    next_generation = copy.deepcopy(generation)
    for index, genetic in enumerate(next_generation):
        if random.random() < Mutation_rate:
            genetic.mutate()

    next_generation.append(copy.deepcopy(best_optim))
    next_generation.append(Momentum())
    next_generation.append(RMSProp())
    
    generation = next_generation[len(next_generation) - Population:]

    # pickleファイルが思ったより容量大きかったので、現世代の評価と進化が終わったら、ひとつ前の世代のpickleファイルを削除
    check_path = folder_path + f"/generation{progress -1}"
    pickle_list = glob.glob(check_path +"/*.pickle")
    for path in pickle_list:
        os.remove(path)

    past_progress = progress

100%|██████████| 50/50 [10:43<00:00, 12.86s/it]


0.5792


100%|██████████| 50/50 [09:57<00:00, 11.95s/it]


0.5792


100%|██████████| 50/50 [09:54<00:00, 11.90s/it]


...

0.5908


100%|██████████| 50/50 [10:27<00:00, 12.54s/it]


0.5908


 86%|████████▌ | 43/50 [09:07<01:29, 12.73s/it]


KeyboardInterrupt: 